# Investigation of an Arrhenius relationship

In addition to being able to determine the mean-squared displacement and diffusion coefficient from a given simulation, `kinisi` also includes [tools](./arrhenius.html) to investigate Arrhenius relationships. 
In this tutorial, we will look at how we can take advantage of these tools to study short, approximately 50 ps, simulations of lithium lanthanum zirconium oxide (LLZO).

<div class="alert alert-warning">

Warning

The warnings that are being ignored are related to the parsing of the files by `MDAnalysis` and lead to unnecessary print out to the screen that we want to avoid in the web documentation.
</div>

In [ ]:
import numpy as np
import scipp as sc
import MDAnalysis as mda
import matplotlib.pyplot as plt
from kinisi.analyze import DiffusionAnalyzer
from kinisi.arrhenius import Arrhenius
import warnings
np.random.seed(42)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

To investigate this we will loop through a series of four temperatures and store each mean and variance of the diffusion coefficient in a dictionary. 

In [ ]:
temperatures = np.array([500, 600, 700, 800])
D = {'mean': [], 'var': []}

To read these simulations we will use [MDAnalysis](https://userguide.mdanalysis.org/stable/index.html) (however, it is also possible to use any `kinisi` parser type).
The parameters are defined for all simulations, here we only consider the diffusive regime to begin after 10 ps.

In [ ]:
params = {'specie': 'LI',
            'time_step': 5.079648e-4 * sc.Unit('ps'),
            'step_skip': 100 * sc.units.dimensionless,
            'progress': False}

File parsing and diffusion determination is then performed in a loop here. 

In [ ]:
for t in temperatures:
    u = mda.Universe(f'_static/traj_{t}.gro', f'_static/traj_{t}.xtc')
    d = DiffusionAnalyzer.from_universe(u, **params)
    d.diffusion(10 * sc.Unit('ps'), progress=False)
    D['mean'].append(sc.mean(d.D).value)
    D['var'].append(sc.var(d.D, ddof=1).value)
    unit = sc.mean(d.D).unit

We can then use the diffusion coefficients and their variances can be used along with the array of temperatures to create a `scipp.DataArray`. 

In [ ]:
td = sc.DataArray(
    data=sc.array(dims=['temperature'], values=D['mean'], unit=unit, variances=D['var']),
    coords={'temperature': sc.Variable(dims=['temperature'], values=temperatures, unit='K')})
td

With the `DataArray` created, we can now pass this to the `Arrhenius` object. 

In [ ]:
s = Arrhenius(td)

This object automatically estimates the maximum likelihood values for the activation energy and preexponential factor and stores them is an appropriate `sc.DataGroup`. 

In [ ]:
s

However, `kinisi` also allows us to estimate the full distribution of these parameters with Markov chain Monte Carlo (MCMC) sampling. 

In [ ]:
s.mcmc()

We can then visualise the probability distributions for the parameters as histograms. 

In [ ]:
from corner import corner

In [ ]:
corner(np.array([i.values for i in s.flatchain.values()]).T, 
       labels=[' / '.join([k, str(v.unit)]) for k, v in s.flatchain.items()])
plt.show()

It is also possible to plot these probability distributions as Arrhenius relations on the data measured values.

In [ ]:
credible_intervals = [[16, 84], [2.5, 97.5], [0.15, 99.85]]
alpha = [0.6, 0.4, 0.2]

plt.errorbar(1000/td.coords['temperature'].values, td.data.values, np.sqrt(td.data.variances), marker='o', ls='', color='k', zorder=10)
for i, ci in enumerate(credible_intervals):
    plt.fill_between(1000/td.coords['temperature'].values,
                     *np.percentile(s.distribution, ci, axis=1),
                     alpha=alpha[i],
                     color='#0173B2',
                     lw=0)
plt.yscale('log')
plt.xlabel('$1000T^{-1}$/K$^{-1}$')
plt.ylabel('$D$/cm$^2$s$^{-1}$')
plt.show()

Finally, the activation energy and preexponential factor are `kinisi.samples.Samples` objects and are stored in the `Arrhenius` object. 

In [ ]:
s.activation_energy

In [ ]:
s.preexponential_factor